

# Nettoyage de la base de données

### <u>1. Description de la base de données

**Les données** </u>

La base de données utilisée est celle de Data.gouv qui liste les festivals en France. Les critères de sélection des festivals sont les suivants :
- avoir eu lieu en 2019
- avoir connu au moins deux éditions en 2019
- se dérouler sur plus d’une journée
- compter au moins 5 spectacles, représentations, concerts ou projections
<u> 



**Description de la base de données** </u>

La base de donnée regroupe les éléments suivants :
- Identité du festival : nom, disciplines dominantes, sous-catégorie au sein des disciplines
- Données géographiques : région de déroulement, département, adresse postale, géocodage xy (coordonnées)
- Données administratives : identifiant, code insee...
- Données temporelles : période de déroulement, date de création...
- Trouver/communiquer avec le festival : site internet et adresse mail.

### <u>2. Suppression des données inutiles 
</u>

On a choisit de supprimer les colonnes suivantes : 
- Code postal (de la commune principale de déroulement)
- Code Insee commune
- Code Insee EPCI
- Libellé EPCI	
- Numéro de voie	 
- Type de voie (rue, Avenue, boulevard, etc.)	
- Nom de la voie	
- Adresse postale	
- Complément d'adresse (facultatif)
- Décennie de création du festival
- Année de création du festival
- Identifiant Agence A
- identifiant CNM

On a aussi fait le choix de ne conserver que les festivals se déroulant en France métropolitaine. 


In [ ]:
import pandas as pd


# Charger le fichier CSV
df = pd.read_csv('festivals_en_France.csv', sep=';', encoding='utf-8-sig')

# Afficher les premières lignes pour vérifier le contenu
print(df.head())


# Supprimer les colonnes qui ne nous intéressent pas
df = df.drop(["Code postal (de la commune principale de déroulement)", "Code Insee commune", "Code Insee EPCI", "Libellé EPCI", "Numéro de voie", "Type de voie (rue, Avenue, boulevard, etc.)", "Nom de la voie", "Adresse postale", "Complément d'adresse (facultatif)", "Décennie de création du festival", "Année de création du festival", "Identifiant Agence A", "identifiant CNM"], axis=1)

# Afficher les colonnes restantes pour vérifier
print(df.columns.tolist())


# Supprimer les festivals hors France métropolitaine
regions_non_metropolitaines = ["Guadeloupe", "Martinique", "Guyane", "La Réunion", "Mayotte", "Polynésie française", "Saint-Pierre-et-Miquelon", "Saint-Barthélemy"]
df = df[~df["Région principale de déroulement"].isin(regions_non_metropolitaines)]

# Vérifier le contenu après suppression
print(df["Région principale de déroulement"].unique())


### <u>3. Modification et mise en forme des données  

**Transformation des données** </u>

Transformation des données sous forme de listes de listes. La première clef utilisée est nom du festival_identifiant car il y a des festivals ayant le même nom mais se déroulant sur des communes différentes. Ajouter l'identifiant permet d'avoir une clef unique. 

On veut quelque chose de cette forme : {"nom-festival_id" : {"nom-festival:..., "date":..., }}, {"nom-festival_id2" : {...} }, etc. 





In [ ]:
# Construire un dictionnaire où chaque clé principale est "Nom du festival_Identifiant"
dictionnaire = {}
for _, row in df.iterrows():
    # Utiliser la colonne "Identifiant" du DataFrame comme partie de la clé
    cle_principale = f"{row['Nom du festival']}_{row['Identifiant']}"
    
    # Ajouter un sous-dictionnaire contenant toutes les colonnes, sauf la colonne "Identifiant"
    dictionnaire[cle_principale] = row.drop("Identifiant").to_dict()

# Afficher un exemple
list(dictionnaire.items())[:5]  # Afficher les 5 premières entrées du dictionnaire pour vérifier


<u>

**La catégorie "pluridisciplinaire"**  </u>

Il s'agit ici d'expliciter la catégorie "pluridisciplinaire". En effet, l'utilisateur va cocher le type de festival qu'il souhaite : il faut donc spécifier à quelles disciplines appartient le festival pour que celui-ci apparaisse lorsque l'utilisateur cochera. Ainsi, si la colonne "Sous-catégorie [...]" n'est pas vide, on renomme la colonne "Discipline dominante" avec le nom de la sous-catégorie.

**Par exemple** : le festival "Rêves d'enfants" est noté pluridisciplinaire et est un festival de littérature et de spectacle vivant. Il faut qu'il apparaisse lorsque l'utilisateur coche "Littérature" ou "Spectacle vivant". On renomme ainsi la colonne "Discipline dominante" "Livre, littérature et Spectacle vivant". 


Il reste néanmoins 290 festivals marqués comme "pluridisciplinaire" mais sans sous-catégorie renseignée. Nous avons donc choisi de supprimer ces festivals pour deux raisons : 
1. Dans le questionnaire, l'utilisateur clique sur une catégorie de festival qu'il souhaite, si le festival n'en a pas il ne peut cliquer dessus. 
2. Il y a trop de festivals dans ce cas (290) pour pouvoir ajouter manuellement les catégories en question. 

**Par exemple**, le festival Printemps de paroles est noté pluridisciplinaire, mais avec aucune sous-catégorie associée.

In [ ]:
# Liste des colonnes sous-catégories que l'on analyse
colonnes_sous_categories = [
    "Sous-catégorie spectacle vivant",
    "Sous-catégorie musique",
    "Sous-catégorie cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques",
    "Sous-catégorie livre et littérature"
]

# Dictionnaire pour associer les colonnes de sous-catégorie à leurs noms dans "Discipline dominante"
mapping_discipline = {
    "Sous-catégorie spectacle vivant": "Spectacle vivant",
    "Sous-catégorie musique": "Musique",
    "Sous-catégorie cinéma et audiovisuel": "Cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques": "Arts visuels, arts numériques",
    "Sous-catégorie livre et littérature": "Livre et littérature"
}

# Fonction pour renommer la discipline dominante en respectant les correspondances spécifiques
def renommer_discipline(row):
    # Si "Discipline dominante" est pluridisciplinaire
    if row["Discipline dominante"] == "Pluridisciplinaire":
        # Liste des disciplines à partir des sous-catégories non vides
        sous_categories = [mapping_discipline[col] for col in colonnes_sous_categories if pd.notna(row[col])]
        # Si au moins une sous-catégorie est trouvée, les utiliser
        if sous_categories:
            return " et ".join(sous_categories)
        else:
            return "A supprimer"
    # Sinon, garder la valeur d'origine
    return row["Discipline dominante"]

# Appliquer la fonction sur le DataFrame
df["Discipline dominante"] = df.apply(renommer_discipline, axis=1)

# Supprimer les festivals marqués "A supprimer"
df = df[df["Discipline dominante"] != "A supprimer"]

# Vérifier les résultats
print(df[["Nom du festival", "Discipline dominante"]].head(50))


<u>

**Modification des sous-catégories** </u>

Pour plus de clarté, nous avons choisis de modifier les sous-catégories présentes dans la base de données qui étaient beaucoup trop nombreuses et les avons réduits à 10 par catégorie. 

**La catégorie Spectacle vivant :**

In [91]:
import re  # Importer le module pour gérer les séparateurs multiples

# Dictionnaire élargi

regroupements_spectacle_vivant = {
    "Théâtre": [
        "Théâtre", "Théâtre - humour", "Théâtre ; Lecture publique", "Arts du théâtre",
        "Théâtre, Danse", "Théâtre, Marionnettes", "Théâtre, arts du conte",
        "Théâtre amateur", "Théâtre de rue", "Arts du théâtre théâtre d'humour",
        "Théâtre musical", "Théâtre - humour comédie", "Théâtre forain", "Du théâtre",
        "Contes", "Représentation", "Poésie", "Lecture…", "Arts du théâtre  théâtre d'humour",
        "Piano jazz théâtre"
    ],
    "Danse": [
        "Danse", "Danses traditionnelles", "Danse contemporaine", "Danse, théâtre",
        "Danses de rue", "Danse afro-contemporaine", "Danse de music’ hall",
        "Danse classique", "Danse theme monde arabe", "Danse theme  europe de l'est",
        "Danse et concerts tango", "Hip hop (danse)", "Capoeira", "Danse contemporaine et danse traditionnelle",
        "Danses", "Bal sévillan"
    ],
    "Arts de la Rue": [
        "Arts de la Rue", "Cirque et Arts de la rue", "Arts de la rue ; Théâtre",
        "Spectacles de rue", "Arts de la rue et Cirque", "Spectacle de rue",
        "Spectacle déambulatoire", "Spectacle de rues", "Arts de la rue et du cirque",
        "Street art", "Arts de la rue - concert - conte", "Des arts de la rue", "Rue",
        "Arts d la rue", "Cultures urbaines ", "Arts de la rue Musique", "Spectacles de rue concerts"
    ],
    "Cirque": [
        "Arts de la piste", "Nouveau Cirque", "Arts du cirque", "Cirque",
        "Clown", "Pyrotechnie", "Magie nouvelle", "Cirque traditionnel", "Mime", "Du cirque",
        "Spectacles équestres", "Arts du crique", " arts du clown"
    ],
    "Musique et Chant": [
        "Musique", "Musiques traditionnelles", "Chanson", "Concerts", "Opéra",
        "Musiques", "Musique ancienne", "Musique médiévale", "Musique et lectures",
        "Chansons théâtrales", "Blues (rythm'n'blues)", "Comédie musicale",
        "Jazz et musiques improvisées", "Musique classique", "Musiques arméniennes",
        "Reggae", "Rock", "Variétés", "Flamenco (musique)", "Musiques du monde",
        "Musiques amplifiées ou électroniques", "Soul funk", "Concerts pop electro",
        "Du chant", "Lecture en musique", "Musiques actuelles", "Musiques traditionnelles de provence",
        " amérique d'avant guerre) des spectacles pour enfants", "Cinéma", "(jazz", " musique savante ",
        " musique savante ", " Chant", " Bal sévillan", "Concours de chants", "Stage musique danse",
        "Rencontres autour de la musique", " Chant", " musique savante ", "Art lyrique ", "Hip-hop ", "Rap", " hip-hop ", " hip-hop / rap"
    ],
    "Marionnettes et Théâtre d'objets": [
        "Théâtre d'objet", "Marionnettes", "Théâtre d'objets", "Marionnettes et Théâtre visuel",
        "Arts de la marionnette", "Marionnettes et théâtre d'objet", "Marionnettes et théâtre d'objets-théâtre d'ombres",
        "Marionnettes-théâtre d'objets-théâtre d'ombres", "Art du mime et du geste", "Marionettes",
        "Marionnettes et théâtre d’objet", "Marionnette", "Marionnettes et théâtre d'objets"
    ],
    "Spectacles pour Jeune Public": [
        "Conte", "Jeune Public", "Conte musical", "Arts du conte",
        "Spectacle pour enfants", "Théâtre jeune public", "Concours de poésie",
        "Découverte des Amériques pour enfants", "Lecture publique", "De la poésie", " decouverte des ameriques pour enfants"
    ],
    "Performance et Arts Visuels": [
        "Performance", "Arts visuels", "photo", "vidéo", "Exposition",
        "Cinéma et audiovisuel", "Magie", "Sculptures", "Parité homme-femme",
        "Pyrotechnie", "Rencontres voyageurs", "Audiovisuel-cinéma", "Ciné-concert",
        "Lecture publique", "Découverte du patrimoine de l’Occitanie autrement",
        "Découverte du Japon", "Culture asiatique", "Thème culture franco colombienne", "theme culture franco colombienne", "La création artistique",
        "L´innovation technologique", "Le recyclage des matériaux", "Le partage des compétences techniques et artistiques",
        "Performance - installation", "Arts", "Rencontrer sur le theme de l'usage du faux et démocratie"
    ],
    "Humour et Café-Théâtre": [
        "Humour", "Théâtre d'humour/café-théâtre", "Café-Théâtre",
        "Théâtre d'humour", "Rires et saveurs: soirée dégustation et concert d'exception", " arts du clown"
    ],
    "Pluridisciplinaire": [
        "Pluridisciplinaire", "Pluridisciplinaire culture", "Pluridisciplinaire à dominante spectacle vivant",
        "Spectacle vivant pluridisciplinaire", "SV hors MUA", "Spectacle vivant",
        "Dégustation de vin", "Thème du Moyen Âge", "Découverte des Amériques pour enfants",
        "Terroir", "Performances", "Toutes les disciplines du spectacle vivant",
        "Conférence", "Solidarités", "Approche de la langue et la culture occitane",
        "Ateliers ouvertures vers l'autres", "Animaux", "Thème du voyage", "Sur le thème du voyage", "Theme du voyage et les sciences",
        "Theme amerique", "Le partage des compétences techniques et artistiques et les Droits Culturels dans le contexte géopolitique transfrontalier suivant les méthodologies de l´Écologie Acoustique.","Marché producteur locaux",
        "Découvrir le patrimoine de l’Occitanie autrement", "Autres (voix d'enfants)", "Animations", "Spectacles", "Et autres…",
        "Des bonnes tablées", "Representation", "Cultures urbaines ", "Bodégas", "Cultures urbaines ", " "
    ]
}

# Inverser le dictionnaire pour une recherche efficace
inverse_regroupements = {}
for categorie, sous_cats in regroupements_spectacle_vivant.items():
    for sous_cat in sous_cats:
        inverse_regroupements[sous_cat.lower().strip()] = categorie

# Fonction pour attribuer les sous-catégories
def attribuer_sous_categories(row):
    if "Spectacle vivant" not in str(row.get("Discipline dominante", "")):
        return None  # Si ce n'est pas du spectacle vivant, ne rien faire
    
    sous_categorie = row.get("Sous-catégorie spectacle vivant")
    if pd.isna(sous_categorie):  # Si la sous-catégorie est NaN
        return None
    
    sous_categories = re.split(r"[;,]", str(sous_categorie))  # Diviser en cas de multiples sous-catégories
    nouvelles_categories = set()
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser : minuscule et sans espaces
        if sous_cat_normalise in inverse_regroupements:
            nouvelles_categories.add(inverse_regroupements[sous_cat_normalise])
        else:
            # Débogage pour les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat}' pour le festival '{row['Nom du festival']}'")
    return list(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction uniquement si Discipline dominante contient "Spectacle vivant"
df["Nouvelles sous-catégories spectacle vivant"] = df.apply(attribuer_sous_categories, axis=1)



In [ ]:
##                  VÉRIFICATION DES RÉSULTATS 

# Filtrer les lignes où "Discipline dominante" contient "Spectacle vivant"
spectacle_vivant_df = df[df["Discipline dominante"].str.contains("Spectacle vivant", na=False)]
    
# Afficher uniquement les colonnes demandées pour les 250 premières lignes
print(spectacle_vivant_df[["Nom du festival", "Région principale de déroulement", "Discipline dominante", "Nouvelles sous-catégories spectacle vivant"]].head(50))


In [ ]:
# Suppression des festivals pour lesquels aucune sous-catégorie n'est renseignée. - il y en a environ 300 sur les 1600 festivals de Spectacle vivant. 

# Filtrer pour conserver uniquement les festivals pour lesquels la discipline dominante contient "Spectacle vivant"
df = df[
    ~(
        df["Discipline dominante"].str.contains("Spectacle vivant", na=False) &  
        (df["Nouvelles sous-catégories spectacle vivant"].isna() |  # La sous-catégorie est NaN
         (df["Nouvelles sous-catégories spectacle vivant"] == "None"))  # La sous-catégorie est "None"
    )
]

##                  VÉRIFICATION DES RÉSULTATS 

# Afficher les résultats pour vérification
print(df[["Nom du festival", "Sous-catégorie spectacle vivant", "Nouvelles sous-catégories spectacle vivant"]].head(20))

# Filtrer les lignes où "Discipline dominante" contient "Spectacle vivant"
spectacle_vivant_df = df[df["Discipline dominante"].str.contains("Spectacle vivant", na=False)]
print(spectacle_vivant_df[["Nom du festival", "Discipline dominante", "Nouvelles sous-catégories spectacle vivant"]].head(50))


Pour faciliter le travail et avoir une idée de la taille des données traitées, on compte ici : 
* Le nombre de festivals ayant pour discipline dominante "Spectacle vivant". 
* Le nombre de festivals dans chaque sous-catégorie. 


In [ ]:
# Compter les lignes où la colonne "Discipline dominante" contient "spectacle vivant"
count_spectacle_vivant = df["Discipline dominante"].str.contains("spectacle vivant", case=False, na=False).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour discipline dominante 'Spectacle vivant' : {count_spectacle_vivant}")


        # THÉÂTRE

# Compter les lignes où la sous-catégorie associée contient "Théâtre"
count_théâtre = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Théâtre" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Théâtre' : {count_théâtre}")


        # DANSE

# Compter les lignes où la sous-catégorie associée contient "Danse"
count_danse = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Danse" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Danse' : {count_danse}")


        # ARTS DE LA RUE 

# Compter les lignes où la sous-catégorie associée contient "Arts de la rue"
count_rue = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Arts de la Rue" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Arts de la rue' : {count_rue}")


        # MUSIQUE ET CHANT 

# Compter les lignes où la sous-catégorie associée contient "Danse"
count_chant = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Musique et Chant" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Musique et Chant' : {count_chant}")


        # CIRQUE

# Compter les lignes où la sous-catégorie associée contient "Cirque"
count_cirque = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Cirque" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Cirque' : {count_cirque}")


        # MARIONNETTES ET THÉÂTRE D'OBJETS

# Compter les lignes où la sous-catégorie associée contient "Marionnettes et Théâtre d'objets"
count_marionnettes = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Marionnettes et Théâtre d'objets" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Marionnettes et Théâtre d'objets' : {count_marionnettes}")
    

    # JEUNE PUBLIC

# Compter les lignes où la sous-catégorie associée contient "Spectacles pour Jeune Public"
count_jeune = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Spectacles pour Jeune Public" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Spectacles pour Jeune Public' : {count_jeune}")


        # PERFORMANCE ET ARTS VISUELS

# Compter les lignes où la sous-catégorie associée contient "Performance et Arts Visuels"
count_visuels = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Performance et Arts Visuels" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Performance et Arts Visuels' : {count_visuels}")

            
        # HUMOUR ET CAFÉ-THÉÂTRE 

# Compter les lignes où la sous-catégorie associée contient "Humour et Café-Théâtre"
count_humour = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Humour et Café-Théâtre" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Humour et Café-Théâtre' : {count_humour}")


        # PLURIDISCIPLINAIRE 

# Compter les lignes où la sous-catégorie associée contient "Pluridisciplinaire"
count_pluri = df["Nouvelles sous-catégories spectacle vivant"].apply(
    lambda x: "Pluridisciplinaire" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Pluridisciplinaire' : {count_pluri}")


**La catégorie Arts visuels, arts numériques :**

In [95]:
# Problèmes dans les sorties "Sous catégorie non reconnue" à cause des virgules dans les parenthèses que le code prend pour un séparateur alors qu'il ne devrait pas. 

import re

def split_with_parentheses_handling(text):
    # Vérifier si le texte est valide
    if not isinstance(text, str) or not text.strip():
        return []
    
    # Diviser en utilisant la regex pour ignorer les virgules dans des parenthèses
    return re.split(r',\s*(?![^(]*\))', text.strip())

In [96]:
import re

# Dictionnaire des regroupements pour Arts Visuels

regroupements_arts_visuels = {
    "Arts numériques et vidéo": [
        "Numérique", "Arts audiovisuels", "Vidéo", "Video : oeuvres multiples d'artistes",
         "Installation numérique", "Jeu vidéo", "Jeux vidéos", 
        "Arts visuels numériques", "Création sur internet", "Vulgarisation", "Formes immersives et interactives", "Art vidéo", "Vidéos d'artistes", "Oeuvres XR (réalité virtuelle et réalité augmentée)", "Arts numériques", "Arts visuels (arts multimédia) ; création contemporaine ; numérique"
    ],

    "Arts plastiques et visuels": [
        "Arts plastiques", "Art plastique", "Sculptures", "Sculpture", "Peinture", "Arts plastiques et visuels",
        "Expositions", "Arts visuels", "Gravure", "Gravures", "Estampes", "Dessin", "Graphisme",
        "Calligraphie et Patrimoine Ecrit", "Art contemporain en général (peinture",
        "Art contemporain au sens large", "Art contemporain et patrimoine", "Expositions d'art contemporain", "Exposition de clocher en clocher", "Art contemporain", "Arts graphiques", "Typographie", "Autres (estampe)", "Céramique", "Vitrail", "Création contemporaine", "Installations artistiques", "Exposition d'art contemporain dans des sites naturels", 
        "Art contemporain en général (peinture, sculpture,...)"
    ],
    
    "Design et architecture": [
        "Design", "Architecture", "Design graphique", "Paysagisme",
        "Mode et design", "Biennale d'architecture et d'urbanisme", 
        "Festival d'architecture", "Festival des Architectures vives", "Paris Design Week", "Festival international de design - design parade Hyères", "Design textile", "Autres : Design", "Autres (textile)", "Art textile",

    ],

    "Arts urbains": [
        "Arts urbains", "Street art", "Graff", 
        "Expériences urbaines", "Arts de la rue", "Graffiti", "Art dans l'espace public",
        "Art en plein air", "In situ", "Bien Urbain, art dans (et avec) l'espace public", "Arts visuel / Art dans l'espace public / Street Art", "Land art", "Parcours Land Art"
    ],

    "Performance et multimédia": [
        "Performance", "Performances", "Installation", "Installations", "Performances multimédias",
        "Ateliers avec les habitants de la ville", "Performances et spectacles hybrides contemporains entre corps et son",
        "Démonstrations", "Rencontres", "Animations diverses", "Performance costume", 
"Performance costume", "Performances multimédias", "Performance - installation", "danse", 'arts de la scène', "Art contemporain ; performance - installation"
    ],

    "Musique et arts sonores": [
        "Musique", "Art sonore et nouvelle musique", "Musiques actuelles",
        "Vinyls", "Nouvelle musique", "Jeune Création", "Street Music", "Arts visuels ; danse ; musique ; théâtre"
    ],

    "Littérature et illustration": [
        "Dessin de presse", "Bande dessinée", "Autre (Dessin)",
        "Calligraphie", "Livres d'artiste", "Salon d'éditions", "Colloque universitaire", "Illustration", "Éditions d'artiste", 'théâtre', 'Théâtre/ danse / musique'
    ],

    "Photographie, cinéma et audiovisuel": [
        "Audiovisuel", "Documentaire", "Vidéos d'artistes", "Programmations croisées avec le cinéma",
        "Cinéma", "Festival d'idées", "Expositions et projections photographie", "Programmations croisées avec le cinéma", "Cinéma et audiovisuel", "Projections documentaires", "Arts visuels (arts multimédia)", "Photographie", "Photo", "Photo exposition à ciel ouvert", "Exposition à ciel ouvert",
        "Photographies", "Street photography", "Vidéo mapping", "Micro édition", "Expositions de photographie", "Photo Montier", "Festival de Street Photography", "Autres (photographie)", 'photographie cinéma rencontres', "Arts visuels ; musiques actuelles ; photographie"
    ],

    "Art d’idées et sciences": [
        "Arts et sciences visionnaires", "Vulgarisation scientifique",
        "Festival d'idées", "Conférence table ronde thème environnement",
        "Festival artistique et citoyen qui questionne notre rapport à l’autre",
        "Sciences et arts", "Exploration des sciences dans les arts"
    ],

    "Autres": [
        "Spectacle vivant", "Culture juive", "Festival artistique et citoyen",
        "Autres : Performances", "Etc.", "Autres (artisans d'art)", "Multiples", "Métiers d'art", "Autres (artisans d'art)", "Autre : Métier d'art de la Céramique",
"Festival des métiers d'art",   "Divers", "Etc.", "Autres (voix d'enfants)", 
        "À l’accueil", "À l’étranger", "Au bien-vivre ensemble dans la diversité", "Pluridisciplinaire", "Autre : Performances"
    ]
}


# Inverser le dictionnaire pour recherche
inverse_regroupements_arts_visuels = {}
for categorie, sous_cats in regroupements_arts_visuels.items():
    for sous_cat in sous_cats:
        inverse_regroupements_arts_visuels[sous_cat.lower().strip()] = categorie

# Fonction pour attribuer les sous-catégories

def attribuer_sous_categories_arts_visuels(row):
    discipline_dominante = str(row.get("Discipline dominante", "")).strip().lower()  # Normalisation
    
    # Vérifier si la discipline dominante contient "arts visuels, arts numériques"
    if "arts visuels, arts numériques" not in discipline_dominante:
        return None

    sous_categorie = row.get("Sous-catégorie arts visuels et arts numériques")
    if pd.isna(sous_categorie):  # Si NaN, retourner None
        return None

    sous_categories = split_with_parentheses_handling(str(sous_categorie))  # Diviser en cas de multiples sous-catégories
    nouvelles_categories = set()
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser
        if sous_cat_normalise in inverse_regroupements_arts_visuels:
            nouvelles_categories.add(inverse_regroupements_arts_visuels[sous_cat_normalise])
        else:
            # Afficher les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat.strip()}' pour le festival '{row['Nom du festival']}'")
    return list(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction
df["Nouvelles sous-catégories arts visuels"] = df.apply(attribuer_sous_categories_arts_visuels, axis=1)



#ajouter base art à la main 

In [ ]:
##                  VÉRIFICATION DES RÉSULTATS 

# Filtrer les festivals pour lesquels la discipline dominante est "Arts visuels, Arts numériques"
arts_visuels_df = df[df["Discipline dominante"].str.contains("Arts visuels", na=False, case=False)]

# Afficher les premières lignes pour vérifier
print(arts_visuels_df[["Nom du festival", "Discipline dominante", "Nouvelles sous-catégories arts visuels"]].head(50))


Pour faciliter le travail et avoir une idée de la taille des données traitées, on compte ici : 
* Le nombre de festivals ayant pour discipline dominante "Arts visuels, arts numériques". 
* Le nombre de festivals dans chaque sous-catégorie. 


In [ ]:
# Compter les lignes où la colonne "Discipline dominante" contient "arts visuels, arts numériques"
count_arts_visuels = df[df["Discipline dominante"].str.contains("Arts visuels, arts numériques", case=False, na=False)].shape[0]

# Afficher le résultat
print(f"Nombre total de festivals avec 'arts visuels et arts numériques' : {count_arts_visuels}")


        # ARTS NUMÉRIQUES ET VIDÉO

# Compter les lignes où la sous-catégorie associée contient "Arts numériques et vidéo"
count_vidéo = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Arts numériques et vidéo" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Arts numériques et vidéo' : {count_vidéo}")


        # ARTS PLASTIQUES ET VISUELS

# Compter les lignes où la sous-catégorie associée contient "Arts plastiques et visuels"
count_plastique = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Arts plastiques et visuels" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Arts plastiques et visuels' : {count_plastique}")


        # DESIGN ET ARCHITECTURE

# Compter les lignes où la sous-catégorie associée contient "Design et architecture"
count_design = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Design et architecture" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Design et architecture' : {count_design}")


        # ARTS URBAINS

# Compter les lignes où la sous-catégorie associée contient "Arts urbains"
count_urbains = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Arts urbains" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Arts urbains' : {count_urbains}")


        # PERFORMANCE ET MULTIMÉDIAS

# Compter les lignes où la sous-catégorie associée contient "Performance et multimédia"
count_perf = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Performance et multimédia" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Performance et multimédia' : {count_perf}")


        # MUSIQUE ET ART SONORE

# Compter les lignes où la sous-catégorie associée contient "Musique et arts sonores"
count_musique = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Musique et arts sonores" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Musique et arts sonores' : {count_musique}")
    

        # PHOTOGRAPHIE, CINÉMA ET AUDIOVISUEL

# Compter les lignes où la sous-catégorie associée contient "Photographie, cinéma et audiovisuel"
count_photo = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Photographie, cinéma et audiovisuel" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Photographie, cinéma et audiovisuel' : {count_photo}")


        # ART D'IDÉES ET SCIENCES

# Compter les lignes où la sous-catégorie associée contient "Art d’idées et sciences"
count_sciences = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Art d’idées et sciences" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Art d’idées et sciences' : {count_sciences}")

            
        # LITTÉRATURE ET ILLUSTRATION

# Compter les lignes où la sous-catégorie associée contient "Littérature et illustration"
count_litt = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Littérature et illustration" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Littérature et illustration' : {count_litt}")


        # AUTRES 

# Compter les lignes où la sous-catégorie associée contient "Autres"
count_autres = df["Nouvelles sous-catégories arts visuels"].apply(
    lambda x: "Autres" in x if isinstance(x, list) else False
).sum()

# Afficher le résultat
print(f"Nombre total de festivals ayant pour sous-catégorie 'Autres' : {count_autres}")


!!! A FAIRE: 

GÉRER LES "NONE"

**La catégorie Cinéma, audiovisuel :**

In [101]:
import re

# Dictionnaire des regroupements pour Cinéma, audiovisuel

regroupements_cinema = {
    "Cinéma généraliste": [
        "Films", "Long métrages", "Court métrages", "Documentaires",
        "Cinéma d'auteur", "Fictions", "Animation", "Ciné-concert"
    ],
    "Audiovisuel et médias": [
        "Audiovisuel", "Télévision", "Webséries", "Contenus digitaux", "Streaming"
    ],
    "Festivals thématiques": [
        "Cinéma de genre", "Cinéma d'horreur", "Cinéma fantastique", "Cinéma LGBTQ+",
        "Cinéma et environnement", "Cinéma et société", "Cinéma et patrimoine"
    ],
    "Cinématographies du monde": [
        "Cinéma européen", "Cinéma asiatique", "Cinéma africain",
        "Cinéma américain", "Cinéma latino-américain"
    ],
    "Rétrospectives et classiques": [
        "Cinéma classique", "Hommages", "Films restaurés",
        "Grands réalisateurs", "Cinéma des années 60", "Cinéma muet"
    ],
    "Techniques et métiers": [
        "Montage", "Réalisation", "Effets spéciaux", "Scénarisation",
        "Photographie de cinéma", "Métiers du cinéma", "Casting"
    ],
    "Cinéma et musique": [
        "Musique de film", "Comédies musicales", "Bande originale",
        "Ciné-opéra", "Ciné-musique"
    ],
    "Cinéma expérimental et arts associés": [
        "Cinéma expérimental", "Vidéos d'artistes", "Art vidéo",
        "Performances visuelles", "Cinéma immersif", "Installation vidéo"
    ],
    "Jeunes publics": [
        "Cinéma jeune public", "Films pour enfants", "Films d'animation",
        "Ciné-contes"
    ],
    "Événements et projections spéciales": [
        "Avant-premières", "Rencontres avec les réalisateurs", "Projections en plein air",
        "Tables rondes", "Ciné-débats", "Projections spéciales"
    ]
}


# Inverser le dictionnaire pour recherche
inverse_regroupements_cinema = {}
for categorie, sous_cats in regroupements_cinema.items():
    for sous_cat in sous_cats:
        inverse_regroupements_cinema[sous_cat.lower().strip()] = categorie

# Fonction pour attribuer les sous-catégories

def attribuer_sous_categories_cinema(row):
    discipline_dominante = str(row.get("Discipline dominante", "")).strip().lower()  # Normalisation
    
    # Vérifier si la discipline dominante contient "Cinéma, audiovisuel"
    if "Cinéma, audiovisuel" not in discipline_dominante:
        return None

    sous_categorie = row.get("Sous-catégorie cinéma et audiovisuel")
    if pd.isna(sous_categorie):  # Si NaN, retourner None
        return None

    sous_categories = re.split(r"[;,]", str(sous_categorie))  # Diviser les sous-catégories multiples
    nouvelles_categories = set()
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser
        if sous_cat_normalise in inverse_regroupements_cinema:
            nouvelles_categories.add(inverse_regroupements_cinema[sous_cat_normalise])
        else:
            # Afficher les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat.strip()}' pour le festival '{row['Nom du festival']}'")
    return list(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction
df["Nouvelles sous-catégories cinéma et audiovisuel"] = df.apply(attribuer_sous_categories_cinema, axis=1)



In [102]:
##                  VÉRIFICATION DES RÉSULTATS 

# Filtrer les festivals pour lesquels la discipline dominante est "Cinéma, audiovisuel"
cinema_df = df[df["Discipline dominante"].str.contains("Cinéma, audiovisuel", na=False, case=False)]

# Afficher les premières lignes pour vérifier
print(cinema_df[["Nom du festival", "Discipline dominante", "Nouvelles sous-catégories cinéma et audiovisuel"]].head(50))


                                       Nom du festival Discipline dominante  \
32                         Festival du cinéma tunisien  Cinéma, audiovisuel   
40                                    Bobines Rebelles  Cinéma, audiovisuel   
77                             International Surf Film  Cinéma, audiovisuel   
79                                            Close-up  Cinéma, audiovisuel   
80                                     Image par Image  Cinéma, audiovisuel   
83                                 A bouches décousues  Cinéma, audiovisuel   
89                         Ciné-rétro en ciné-concerts  Cinéma, audiovisuel   
90                          Festival du film d'un jour  Cinéma, audiovisuel   
93                          Le Festival du fantastique  Cinéma, audiovisuel   
97   Rencontres Cinématographiques La Classe Ouvriè...  Cinéma, audiovisuel   
98                 Soirées du Cinéma Russe de Bordeaux  Cinéma, audiovisuel   
99                                La balade des cour

NE MARCHE PAS, À REVOIR

**La catégorie Livre, littérature**

In [103]:
# Problèmes dans les sorties "Sous catégorie non reconnue" à cause des virgules dans les parenthèses que le code prend pour un séparateur alors qu'il ne devrait pas. 

import re

def split_with_parentheses_handling(text):
    # Vérifier si le texte est valide
    if not isinstance(text, str) or not text.strip():
        return []
    
    # Diviser en utilisant la regex pour ignorer les virgules dans des parenthèses
    return re.split(r',\s*(?![^(]*\))', text.strip())


In [104]:
import re  # Importer le module pour gérer les séparateurs multiples

# Dictionnaire élargi

regroupements_livre_litterature = {
    "Romans et Littérature Générale": [
        "Romans", "Littérature", "Essais", "Biographies", "Autobiographies", 
        "Poésie", "Recueils", "Nouvelles", "Textes littéraires", "essai", " autobiographie", "Littérature générale", "théâtre", "Fiction (roman, théâtre, etc.)", "Fiction", "Conte",
        "Non-fiction (documentaire, autobiographie, essai, récit, etc.)", "Généraliste", "Littérature et écriture contemporaine", 
        "Voyage", "Fictions", "Non fiction", "Tout les genres",
        "Littérature contemporaine", "Rentrée littéraire littérature générale", "Création littéraire",
"Littérature blanche", "Fictions (romans, théâtre, etc.)",
        "Fictions (roman, théâtre, etc.) Non-fiction (documentaire, autobiographie, essai, récit, etc.)",    "Actualité littéraire", "Livre ancien",      "Poésie contemporaine", "Oulipo", "Littérature performative"

    ],
    "Bandes Dessinées et Illustrations": [
        "Bandes dessinées", "Mangas", "Comics", "Illustrations", "Graphic novels", 
        "Dessins de presse", "Albums illustrés", "manga", " Bande dessinée", "Bande-dessinée", "Livre d'artistes", "livre d'art", "livres d'artistes",
        "micro-édition et images imprimées", "animé", "Bande dessiné", "Bande déssinée", "BD et livres jeunesse", "Rencontres du 9e art", "Zébuli salon de illustration et BD jeunesse", "Art lyrique ; bandes dessinées", "Manga et culture asiatique",         "Livre d’artiste et image imprimée",         

    ],
    "Jeunesse et Jeune Public": [
        "Littérature jeunesse", "Contes", "Livres pour enfants", "Histoires pour jeunes", 
        "Albums jeunesse", "Contes musicaux", "Ciné-contes", "Jeunesse", " jeunesse", "Littérature exclusivement jeunesse", "Jeune public", "Littérature jeunesse en général", "Albums et illustrations jeunesse",
        "Littérature jeunesse - petite enfance", "Album jeunesse", "Enfance jeunesse",
        "Petite enfance", "Raconte bébés", "Festival jeune public et famille"

    ],
    "Policier et Thriller": [
        "Littérature policière", "Thrillers", "Romans noirs", "Mystères", "Enquêtes", "Polar", "Policier"
    ],
    "Science-Fiction et Fantasy": [
        "Science-fiction", "Fantasy", "Fantastique", "Sagas", "Univers imaginaires", 
        "Épopées", "Steampunk", "Imaginaire",     "Science fiction", "SF", "Épouvante"
    ],
    "Littératures régionales et du Monde": [
        "Littérature étrangère", "Littératures européennes", "Littératures asiatiques", 
        "Littératures africaines", "Littératures américaines", "Traditions orales", "littérature régionale", "Littérature-monde", "Russophonie et francophonie", "Carnet et littérature de voyage", "Histoire - mer - aventures etc", "Récits de voyage"


    ],
    "Édition et Métiers du Livre": [
        "Édition", "Auto-édition", "Librairies", "Ateliers d'écriture", 
        "Typographie", "Illustrations éditoriales", "Métiers du livre", "Livres d'artistes / Petite édition", "Éditions d'art, livres d'artistes"

    ],
    "Conférences et Rencontres Littéraires": [
        "Rencontres avec auteurs", "Lectures publiques", "Conférences", "Ateliers littéraires", 
        "Échanges littéraires", "Tables rondes", "Dédicaces", "Eloquence", "lecture", "Lecture à voix haute", "Traduction littéraire", "littérature de critique sociale",
        "philosophie", "lecture à voix haute", "salon du livre", "Expositions", "Rencontres", "Salon de rencontre des auteurs de théâtre et cinéma", "Rencontres littéraires",         "Lectures", "Lectures musicales", "Performances"
    ],

    "Histoire et Patrimoine Littéraire": [
        "Archives", "Histoire du livre", "Manuscrits", "Bibliothèques historiques", 
        "Littératures classiques", "Récits historiques", "Textes anciens", "Livre ancien",         "Histoire", "Histoire et patrimoine", "Histoire et sciences humaines", 
    ],

    "Pluridisciplinaire : arts et littératures croisés ": [
        "Pluridisciplinaire", "Livre et arts visuels", "Livres et vin", "Art lyrique ; bandes dessinées ; musique savante ; audiovisuels", "Art",         "Art lyrique ; musique savante", "Musique", "Art lyrique ; musiques actuelles ; théâtre",
        "Art lyrique ; danse modern jazz ; jazz et musiques improvisées ; musique savante ; théâtre",
    "Art lyrique ; jazz ; musique savante ; musiques de films", "Livre et vin"
        "Art lyrique ; jazz ; jazz et musiques improvisées ; musique classique", "Art lyrique ; jazz ; jazz et musiques improvisées ; musique (d'harmonie) ; musique classique ; musique savante ; musiques traditionnelles", "Tous genres littéraires et animations dépendant de la thématique",         "Littérature scientifique", "Sciences humaines", "Sociales", "Lettres", "Sciences fondamentales", "Développement personnel", "Bien-être", "Pas de sous-catégorie", "Toutes thématiques"

    
]
}


# Inverser le dictionnaire pour une recherche efficace
inverse_regroupements = {}
for categorie, sous_cats in regroupements_livre_litterature.items():
    for sous_cat in sous_cats:
        inverse_regroupements[sous_cat.lower().strip()] = categorie

# Fonction pour attribuer les sous-catégories
def attribuer_sous_categories(row):
    if "Livre" not in str(row.get("Discipline dominante", "")):
        return None  # Si ce n'est pas du spectacle vivant, ne rien faire
    
    sous_categorie = row.get("Sous-catégorie livre et littérature")
    if pd.isna(sous_categorie):  # Si la sous-catégorie est NaN
        return None
    
    sous_categories = split_with_parentheses_handling(str(sous_categorie))  # Diviser en cas de multiples sous-catégories
    nouvelles_categories = set()
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser : minuscule et sans espaces
        if sous_cat_normalise in inverse_regroupements:
            nouvelles_categories.add(inverse_regroupements[sous_cat_normalise])
        else:
            # Débogage pour les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat}' pour le festival '{row['Nom du festival']}'")
    return list(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction uniquement si Discipline dominante contient "Spectacle vivant"
df["Nouvelles sous-catégories livre et littérature"] = df.apply(attribuer_sous_categories, axis=1)




# Citéphilo à ajouter à la main ; formule BUla 


Sous-catégorie non reconnue : 'bibliophilie' pour le festival 'Microscopies'
Sous-catégorie non reconnue : 'littérature générale et littérature jeunesse' pour le festival 'Rencontres d'été théâtre & lecture en Normandie'
Sous-catégorie non reconnue : 'Animations autour de la langue gallèse' pour le festival 'Mil Goul'
Sous-catégorie non reconnue : 'livre audio' pour le festival 'Festival du Livre Audio et du Podcast'
Sous-catégorie non reconnue : 'tous les genres si marseille est présente dans le livre' pour le festival 'Carré des écrivains'
Sous-catégorie non reconnue : 'Culture' pour le festival 'Semaine de la pop philosophie'
Sous-catégorie non reconnue : 'Fiction (roman, théâtre, etc.) Non-fiction (documentaire, autobiographie, essai, récit, etc.)' pour le festival 'Quais du départ'
Sous-catégorie non reconnue : 'Littératures et Pratiques Culturelles' pour le festival 'Le Printemps des Poètes'
Sous-catégorie non reconnue : 'Tous genres littéraires' pour le festival 'Mois du livre'


In [ ]:
##                  VÉRIFICATION DES RÉSULTATS 

# Filtrer les festivals pour lesquels la discipline dominante est "Cinéma, audiovisuel"
livre_df = df[df["Discipline dominante"].str.contains("Livre, littérature", na=False, case=False)]

# Afficher les premières lignes pour vérifier
print(livre_df[["Nom du festival", "Discipline dominante", "Nouvelles sous-catégories livre et littérature"]].head(50))
